# LLM powered Chatbot
- Design and Implement a LLM powered ChatBot.

- It will be able to have conversation and remember previous interaction

### Related Topics: 
1. **Conversational RAG**: ChatBot experience over **external source of data**.

2. **Agents**: ChatBot that can take **Action**.

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHINA_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROKECT"] = "LLM_POWERED_CHATBOT"


In [5]:
from langchain_groq import ChatGroq
llm = ChatGroq(model = "gemma2-9b-it")

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    HumanMessage(content = "Hi, my name is Sujit and I am studying Gen AI"),
    AIMessage(content = "Hello Sujit! It's nice to meet you. \n\nWhat field in Gen AI are you studying these days? \n\nIm always eager to aid you in your studies, let me know if you need something."),
    HumanMessage(content = "Hey what is my name and what do I study?")
]

llm.invoke(messages)

AIMessage(content="You are Sujit, and you are studying Gen AI. 😊 \n\nIs there anything else you'd like to know or discuss about Gen AI? I'm happy to help in any way I can.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 88, 'total_tokens': 134, 'completion_time': 0.083636364, 'prompt_time': 0.0054058, 'queue_time': 0.26442536, 'total_time': 0.089042164}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--65bad498-8f84-47f9-be5c-5c06017e84e9-0', usage_metadata={'input_tokens': 88, 'output_tokens': 46, 'total_tokens': 134})

### Message History

- We can use a `Message history class` to wrap our model and make it `stateful`. <br>

- This will keep track of `inputs and outputs` of our model and store them in a `datastore`.<br>

- Future interactions will then `load those messages` and pass them into chain as `part of the input`. <br>



In [7]:
# import relevant class, set up our chain which wraps the model and add in the message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [8]:
store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [9]:
config = {"configurable" : {"session_id" : "Chat1"}}

In [10]:
with_message_history = RunnableWithMessageHistory(llm, get_session_history)

response = with_message_history.invoke(
    [HumanMessage(content = "Hi, my name is Sujit and I am studying Gen AI")],
    config = config
)

response.content

"Hi Sujit,\n\nIt's great to meet you!\n\nGen AI is a fascinating field. What aspects of it are you most interested in? \n\nPerhaps I can help you with some information or answer any questions you have. 😊\n"

In [11]:
with_message_history.invoke(
    [HumanMessage(content = "What's my name and what do i study?")],
    config = config
)

AIMessage(content="You told me your name is Sujit and you are studying Gen AI!  \n\nIs there anything else you'd like to know or discuss about it? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 93, 'total_tokens': 130, 'completion_time': 0.067272727, 'prompt_time': 0.00512947, 'queue_time': 0.265239668, 'total_time': 0.072402197}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b44da66f-230a-494e-a590-03b3fb6c8857-0', usage_metadata={'input_tokens': 93, 'output_tokens': 37, 'total_tokens': 130})

In [12]:
# changing the session id
config1 = {"configurable" : {"session_id" : "Chat2"}}

with_message_history.invoke(
    [HumanMessage(content = "What's my name?")],
    config = config1
)

AIMessage(content="As an AI, I have no memory of past conversations and no way of knowing your name. If you'd like to tell me your name, I'd be happy to use it!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 15, 'total_tokens': 57, 'completion_time': 0.076363636, 'prompt_time': 0.001848747, 'queue_time': 0.258536541, 'total_time': 0.078212383}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--80685112-4175-41ab-b1aa-c448c06f6e78-0', usage_metadata={'input_tokens': 15, 'output_tokens': 42, 'total_tokens': 57})

By changing `session_id` we can switch the context of conversation we are having.

### Prompt Template:
Integrating prompt templates in ChatBots.

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer the questions to the best of your capability."),
        MessagesPlaceholder(variable_name = "messages")
    ]
)

chain = prompt|llm

In [14]:
chain.invoke({"messages":[HumanMessage(content = "Hi, my name is Sujit")]})

AIMessage(content="Hello Sujit! It's nice to meet you. \n\nHow can I help you today? 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 32, 'total_tokens': 59, 'completion_time': 0.049090909, 'prompt_time': 0.002281435, 'queue_time': 0.257851255, 'total_time': 0.051372344}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f34c17dd-bc8c-4372-8a5e-c1752b95f7f3-0', usage_metadata={'input_tokens': 32, 'output_tokens': 27, 'total_tokens': 59})

In [15]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [16]:
config2 = {"configurable" : {"session_id" : "Chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content = "Hi, my name is Sujit")],
    config = config2
)

response.content

"Hello Sujit! It's nice to meet you.  \n\nHow can I help you today? 😊  \n\n"

In [17]:
# adding more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer the questions to the best of your capability in {language}."),
        MessagesPlaceholder(variable_name = "messages")
    ]
)

chain = prompt|llm 

In [18]:
response = chain.invoke(
    {
        "messages": [HumanMessage(content = "Hi, my name is sujit.")],
        "language": "Hindi"
    }
)
response.content

'नमस्ते सुजित! मैं आपकी मदद करने के लिए तैयार हूँ। आप मुझसे क्या पूछना चाहते हैं? \n'

In [19]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key= "messages"
)

In [20]:
config3 = {"configurable": {"session_id": "Chat4"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content= "Hi, I am Sujit")], "language": "Marathi"},
    config = config3
)

response.content

'नमस्कार सुजित! मला आनंद आहे तुम्हाला मदत करण्यासाठी. तुम्हाला काय प्रश्न आहे? \n'

### Managing Conversation History
1. LLMs have a fixed **context size**, whereas **Chat Histories** may not

2. If left unmanaged, list of messages will **grow unbounded** 

3. Potentially **overflow** the **Context window** of LLM

It is Important to add a step that `limits the size of messages` you are passing in.

In [21]:
from langchain_core.messages import trim_messages


`trim_messages` helps reduce how many messages we're sending to the model. <br>

It allows us to specify how many tokens we want to keep, along with other parameters. <br>

In [30]:
trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last", # focuses on the last conversation it had
    token_counter = llm, 
    include_system = True, # include the system message
    allow_partial = False, # dont allow partial message
    start_on = "human" # it should start from human conversation
)

messages = [
    SystemMessage(content= "You're a good assistant"),
    HumanMessage(content= "Hi! I am Rob"),
    AIMessage(content= "Hi!"),
    HumanMessage(content= "I like Chocolate Ice cream"),
    AIMessage(content= "nice"),
    HumanMessage(content= "what's 2+2"),
    AIMessage(content= "4"),
    HumanMessage(content= "Thanks!"),
    AIMessage(content= "no problem"),
    HumanMessage(content= "Having fun?"),
    AIMessage(content= "yes"),
]

trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi! I am Rob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like Chocolate Ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's 2+2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes', additional_kwargs={}, response_metadata={})]

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages")|trimmer)
    | prompt 
    | llm
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content= "What ice-cream do i like?")],
        "language":"English"
    }
)

response.content

'You said you like chocolate ice cream!  🍦😊  Anything else I can help you with? \n'

In [32]:
# Lets wrap this in message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key= "messages",
)
config = {"configurable":{"session_id":"FinalChat"}}